In [1]:
import gtfstk as gt
import pandas as pd
import geopandas as gpd
import numpy as np
import tkinter as tk
import requests, zipfile, io, os
from tkinter import filedialog

In [2]:
def button_gtfs_clicked():
    root.gtfs=filedialog.askopenfilename(initialdir="/",title="Select GTFS Zip file",filetypes = (("zip files","*.zip"),("all files","*.*")))
def button_outfolder_clicked():
    root.outfolder=filedialog.askdirectory(initialdir="/",title="Select Output folder")
def close_window():
    root.destroy()

root=tk.Tk()
root.title('Stops Changes')


label_gtfs=tk.Label(root,text="Select GTFS zip file.")
label_gtfs.grid(column=0,row=0)
button_gtfs=tk.Button(root,text="Browse",command=button_gtfs_clicked)
button_gtfs.grid(column=0,row=1)

label_out=tk.Label(root,text="Select an output folder")
label_out.grid(column=0,row=2)
button_out=tk.Button(root,text="Browse",command=button_outfolder_clicked)
button_out.grid(column=0,row=3)

button_run=tk.Button(root,text='Run',width=25,command=close_window)
button_run.grid(column=0,row=4)
root.mainloop()

In [3]:
zip_file_url=r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("gtfs")
gtfs_filename=os.path.basename(zip_file_url)
gtfs_filename=os.path.splitext(gtfs_filename)[0]
old=gt.read_gtfs("gtfs",dist_units='mi')
new=gt.read_gtfs(root.gtfs,dist_units='mi')
gtfs_filename=os.path.basename(root.gtfs)
gtfs_filename=os.path.splitext(gtfs_filename)[0]+"_stop_changes"

In [4]:
outfile=root.outfolder+"\{}.xlsx"

In [8]:
def getRoutes(feed):
    feed.routes['route_short_name']=feed.routes['route_short_name'].str.replace('R',"",regex=True)
    df=feed.stop_times.merge(feed.trips)
    df=df.merge(feed.routes)
    df=df.groupby(['stop_id','route_short_name'],as_index=False).first()
    df=pd.DataFrame(df.groupby('stop_id')['route_short_name'].agg(lambda x: ', '.join(x))).reset_index()
    df=df[['stop_id','route_short_name']]
    return df

In [13]:
def compareFeeds(old,new,save_file=None):
    old_feed=getRoutes(old)
    new_feed=getRoutes(new)
    df=old_feed.merge(new_feed,on='stop_id',suffixes=('_old','_new'),how='outer')
    df['Status']='No Change'
    df['Status'][df['route_short_name_old']!=df['route_short_name_new']]='Decal Change'
    df['Status'][df['route_short_name_old'].isna()]='New Stop'
    df['Status'][df['route_short_name_new'].isna()]='Close Stop'
    df=df.merge(new.stops,how='left',on='stop_id',suffixes=("",'_new'))
    df=df.merge(old.stops,how='left',on='stop_id',suffixes=("",'_old'))
    df['Stop Name']=df['stop_name']
    df['Stop Name'][df['stop_name'].isna()]=df['stop_name_old']
    df.rename(columns={'route_short_name_old':'Old Routes','route_short_name_new':'New Routes'},inplace=True)
    df=df[['stop_id','Stop Name','Old Routes','New Routes','Status']]
    df.sort_values('Status',inplace=True)
    if save_file:
        df.to_excel(save_file,index=False)
    return df
compareFeeds(old,new,outfile.format(gtfs_filename))

c:\users\bparker\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,stop_id,Stop Name,Old Routes,New Routes,Status
545,01011029,State & Mercer NEM,9,NaN,Close Stop
560,01011052,Chinden @ Hewlett Packard,"8, 8x",NaN,Close Stop
559,01011051,Vista & Canal SEC,3,NaN,Close Stop
558,01011050,Parkcenter & Barber Valley NWC,1,NaN,Close Stop
557,01011049,Parkcenter & Barber Valley SEC,1,NaN,Close Stop
556,01011048,Millspur Loop,1,NaN,Close Stop
555,01011041,State & Arthur SWC,9,NaN,Close Stop
554,01011040,State & Northgate Shopping Center,9,NaN,Close Stop
93,01010298,Five Mile & Emerald SEC,"8, 8x",NaN,Close Stop
553,01011038,Emerald & Liberty NWC,5,NaN,Close Stop
